In [2]:
# Install dependencies for RSS fetching, NLP processing, and summarization
!pip install requests beautifulsoup4 lxml spacy transformers markdown scikit-learn pandas sqlite3
!python -m spacy download en_core_web_md


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     ----- ---------------------------------- 5.8/42.8 MB 29.3 MB/s eta 0:00:02
     ----------- --------------------------- 12.3/42.8 MB 30.8 MB/s eta 0:00:01
     ----------------- --------------------- 19.1/42.8 MB 30.9 MB/s eta 0:00:01
     ------------------- ------------------- 21.2/42.8 MB 30.5 MB/s eta 0:00:01
     -------------------- ------------------ 22.3/42.8 MB 22.7 MB/s eta 0:00:01
     --------------------- ----------------- 24.1/42.8 MB 19.6 MB/s eta 0:00:01
     ------------------------ -------------- 26.7/42.8 MB 18.6 MB/s eta 0:00:01
     ---------------------------- ---------- 31.2/42.8 MB 19.0 MB/s eta 0:00:01
     ------------------------------- ------- 34.1/42.8 MB 18.3 MB/s eta 0:00:01
     ----------------------------------- --- 39.3/42.8 MB 19.1 MB/s eta 0:00:01
     --------------------------------------  42.7/42.8 MB 19.6 MB/s eta 0:00:01
     --------------------------------------- 42.

2025-03-23 15:20:33.624986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 15:20:37.385665: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
### STEP 2: Import Necessary Libraries
import requests
from bs4 import BeautifulSoup
import spacy
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from IPython.core.display import display, Markdown, HTML
import json
import os

C:\Users\91996\AppData\Local\Temp\ipykernel_17832\3043698543.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, Markdown, HTML


In [4]:
### STEP 3: Initialize NLP Models
nlp = spacy.load("en_core_web_md")  # Medium model with word vectors
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

C:\Users\91996\anaconda3\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91996\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [42]:
### STEP 4: Setup User Preferences Database (SQLite)
DB_FILE = "flashfeed_users.db"
def setup_database():
    """Creates a user preferences database if not exists."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS user_preferences (
                        user TEXT PRIMARY KEY, 
                        interests TEXT, 
                        last_read_articles TEXT)''')
    conn.commit()
    conn.close()
setup_database()

In [43]:
### STEP 5: Define User Interest Learning Function
def update_user_interests(user, new_interests):
    """Updates user preferences in the database dynamically."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT interests FROM user_preferences WHERE user=?", (user,))
    result = cursor.fetchone()
    if result:
        existing_interests = set(json.loads(result[0]))
        updated_interests = list(existing_interests.union(set(new_interests)))
        cursor.execute("UPDATE user_preferences SET interests=? WHERE user=?", (json.dumps(updated_interests), user))
    else:
        cursor.execute("INSERT INTO user_preferences (user, interests) VALUES (?, ?)", (user, json.dumps(new_interests)))
    conn.commit()
    conn.close()


In [44]:
def fetch_rss_articles(rss_urls):
    """Fetches and parses news articles from RSS feeds without printing warnings."""
    articles = []
    for url in rss_urls:
        try:
            response = requests.get(url, timeout=5)
            if response.status_code != 200:
                continue  # Silently skip broken RSS feeds

            soup = BeautifulSoup(response.content, "xml")
            for item in soup.find_all("item")[:5]:  # Fetch first 5 articles per feed
                title = item.title.text if item.title else "No Title"
                link = item.link.text if item.link else "No Link"
                summary = item.description.text if item.description else "No Summary"
                published = item.pubDate.text if item.pubDate else "No Date"

                articles.append({
                    "title": title,
                    "link": link,
                    "summary": summary,
                    "published": published
                })

        except requests.exceptions.RequestException:
            pass  # Suppress error messages

    return articles


In [45]:
def extract_keywords(text):
    """Extracts important keywords using NLP and adds more nouns for better filtering."""
    doc = nlp(text)
    keywords = set()

    # Extract Named Entities
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT", "GPE", "WORK_OF_ART", "TECHNOLOGY"]:
            keywords.add(ent.text.lower())

    # Extract Key Nouns, Proper Nouns, and Common Tech Words
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:
            keywords.add(token.text.lower())

    # Include some common tech-related words manually
    common_tech_terms = {"ai", "machine learning", "deep learning", "cybersecurity", "blockchain", "startup"}
    keywords.update(common_tech_terms)

    return keywords


In [46]:
def filter_relevant_articles(articles, user_interests):
    """Filters articles to ensure they match user interests based on improved NLP categorization."""
    relevant_articles = []

    for article in articles:
        article_text = (article["title"] + " " + article["summary"]).lower()
        category = categorize_article(article_text)

        # Allow articles if at least one keyword or category matches
        match_count = sum(1 for interest in user_interests if interest.lower() in category.lower() or interest.lower() in article_text)

        if match_count >= 1:  # Loosen the filtering rule
            relevant_articles.append(article)
    
    return relevant_articles


In [47]:
### STEP 7: Categorize Articles Using NLP
def categorize_article(article_text):
    """Categorizes an article based on its semantic meaning using word vector similarity."""
    categories = {
        "Technology": "AI cybersecurity blockchain startups programming machine learning software gadgets data science robotics cloud computing chip manufacturing open-source innovation",
        "Finance": "markets fintech cryptocurrency economics trading stocks investments banking financial analysis inflation",
        "Sports": "Football F1 NBA Olympics esports tennis cricket rugby athletics soccer racing World Cup Champions League",
        "Entertainment": "Movies TV shows music books celebrities film industry Hollywood Netflix Bollywood",
        "Science": "Space exploration biotech physics renewable energy genetics NASA climate change quantum computing astronomy"
    }
    
    doc = nlp(article_text.lower())
    highest_score = 0
    best_category = "Uncategorized"
    
    for category, keywords in categories.items():
        category_doc = nlp(keywords)
        similarity = doc.similarity(category_doc)
        # Increase threshold for better accuracy
        if similarity > highest_score and similarity > 0.5:  
            highest_score = similarity
            best_category = category
    
    return best_category

In [48]:
### STEP 8: Summarize Articles Using AI
def summarize_articles(articles):
    """Summarizes articles using NLP-based text summarization."""
    summarized_articles = []
    for article in articles:
        try:
            short_summary = summarizer(article["summary"], max_length=50, min_length=20, do_sample=False)
            article["summary"] = short_summary[0]["summary_text"]
        except Exception as e:
            print(f"⚠️ Error summarizing article: {e}")
        summarized_articles.append(article)
    return summarized_articles

In [49]:
### STEP 9: Generate Personalized Newsletters
def generate_newsletter(user, articles):
    """Generates a personalized Markdown newsletter."""
    newsletter_content = f"# FlashFeed: Personalized Newsletter for {user}\n\n"
    newsletter_content += "## Top Stories\n\n"
    for article in articles:
        newsletter_content += f"### {article['title']}\n"
        newsletter_content += f"📅 {article['published']}\n\n"
        newsletter_content += f"**Summary:** {article['summary']}\n\n"
        newsletter_content += f"[Read more]({article['link']})\n\n"
        newsletter_content += "---\n"
    file_name = f"{user.replace(' ', '_').lower()}_flashfeed.md"
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(newsletter_content)
    return file_name


In [50]:
def generate_newsletters_for_all():
    """Fetches, filters, and generates newsletters for all users dynamically."""
    users = {
        "Alex Parker": {
            "interests": ["AI", "cybersecurity", "blockchain", "startups", "programming"],
            "sources": ["https://techcrunch.com/feed/", "https://www.wired.com/feed/rss"]
        },
        "Priya Sharma": {
            "interests": ["Global markets", "startups", "fintech", "cryptocurrency", "economics"],
            "sources": ["https://www.bloomberg.com/feeds/latest.xml", "https://www.forbes.com/finance/feed/"]
        },
        "Marco Rossi": {
            "interests": ["Football", "F1", "NBA", "Olympic sports", "esports"],
            "sources": ["https://www.espn.com/espn/rss/news", "https://www.skysports.com/rss/12040"]
        },
        "Lisa Thompson": {
            "interests": ["Movies", "celebrity news", "TV shows", "music", "books"],
            "sources": ["https://variety.com/feed/", "https://www.billboard.com/rss.xml"]
        },
        "David Martinez": {
            "interests": ["Space exploration", "AI", "biotech", "physics", "renewable energy"],
            "sources": ["https://www.nasa.gov/rss/dyn/breaking_news.rss", "https://www.sciencedaily.com/rss/all.xml"]
        }
    }

    for user, data in users.items():
        articles = fetch_rss_articles(data["sources"])  # Fetch only user-specific sources
        filtered_articles = filter_relevant_articles(articles, data["interests"])
        summarized_articles = summarize_articles(filtered_articles)
        file_name = generate_newsletter(user, summarized_articles)
        print(f"📩 Newsletter created for {user}: {file_name}")

generate_newsletters_for_all()


Your max_length is set to 50, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 50, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 50, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


📩 Newsletter created for Alex Parker: alex_parker_flashfeed.md
📩 Newsletter created for Priya Sharma: priya_sharma_flashfeed.md
📩 Newsletter created for Marco Rossi: marco_rossi_flashfeed.md
📩 Newsletter created for Lisa Thompson: lisa_thompson_flashfeed.md
📩 Newsletter created for David Martinez: david_martinez_flashfeed.md


In [23]:
from IPython.core.display import display, Markdown

def display_newsletter(file_name):
    """Reads and displays the generated Markdown newsletter inside Jupyter Notebook."""
    try:
        with open(file_name, "r", encoding="utf-8") as f:
            content = f.read()
        
        display(Markdown(content))
    except FileNotFoundError:
        print(f"⚠️ File not found: {file_name}. Make sure the newsletter was generated successfully.")

# Example usage (Check Alex Parker's feed)
display_newsletter("alex_parker_flashfeed.md")



C:\Users\91996\AppData\Local\Temp\ipykernel_17832\3679544274.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, Markdown


# FlashFeed: Personalized Newsletter for Alex Parker

## Top Stories

### Jonah Peretti helped shaped digital media — can he do it again?
📅 Sun, 23 Mar 2025 01:00:52 +0000

**Summary:** Jonah Peretti is the founder and CEO of BuzzFeed. Peretti has been at the forefront of digital media for almost two decades. As the company grows older, one question lingers: What is next?

[Read more](https://techcrunch.com/2025/03/22/jonah-peretti-helped-shaped-digital-media-can-he-do-it-again/)

---
### Charlie Javice trial becomes a master class in hubris for both sides
📅 Sat, 22 Mar 2025 20:49:25 +0000

**Summary:** Charlie Javice’s fraud trial has become a showcase of embarrassing missteps on both sides. JPMorgan Chase was allegedly deceived into buying her startup, Frank, for $175 million when it had just 300,000 customers.

[Read more](https://techcrunch.com/2025/03/22/charlie-javice-trial-becomes-a-master-class-in-hubris-for-both-sides/)

---
### Week in Review: Google buys Wiz 
📅 Sat, 22 Mar 2025 17:05:00 +0000

**Summary:** Google made its biggest acquisition in its history this week when it bought Wiz. The NASA astronauts finally came home; Rippling settled its lawsuit.

[Read more](https://techcrunch.com/2025/03/22/week-in-review-google-buys-wiz/)

---
### The 20 hottest open source startups of 2024
📅 Sat, 22 Mar 2025 14:00:00 +0000

**Summary:** A new report showcases the 20 top-trending open source startups around the world. The report is the handiwork of European venture capital firm Runa Capital.

[Read more](https://techcrunch.com/2025/03/22/the-20-hottest-open-source-startups-of-2024/)

---
### Meta settles UK ‘right to object to ad-tracking’ lawsuit by agreeing not to track plaintiff
📅 Sat, 22 Mar 2025 00:01:00 +0000

**Summary:** Human rights campaigner Tanya O&#8217;Carroll has succeeded in forcing social media giant Meta not to use her data for targeted advertising. The agreement is contained in a settlement to an individual challenge she lodged against Meta&#

[Read more](https://techcrunch.com/2025/03/21/meta-settles-u-k-right-to-object-to-ad-tracking-lawsuit-by-agreeing-not-to-track-plaintiff/)

---
